In [12]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
import itertools
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from prophet.diagnostics import cross_validation, performance_metrics
from suntime import Sun, SunTimeException
from datetime import timedelta
import ephem
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import r2_score
import mlflow

In [2]:
df=pd.read_csv("/workspaces/Powerbox/timeseries.csv")
df.head()

,Unnamed: 0,Timestamp,System ON,System ON Timestamps,System OFF Timestamps,Temperature (°C),Solar Panels Energy Output (W),Power Consumption (kW),Energy Stored in Batteries (kWh),Inverter Efficiency (%),...,Battery Full Flag,Customer Profile,User Coordinates,Solar Panels Type,Solar Panels Configuration,Depth of Discharge,Battery Capacity (Wh),Inverter Capacity (kW),Battery Technology,Solar Irradiance (W/m²)
0,1,05/09/2024 00:00,True,05/09/2024 00:00,NaN,21.027232,0.0,0.232676,0.767498,90.0,...,False,Residential,"10.525053,7.417248",MonoCrystalline,1s2p,10%,1000.0,1.0,Lithium - Ion,NaN
1,3,05/09/2024 00:15,True,05/09/2024 00:15,NaN,21.000000,0.0,0.398544,0.709329,90.0,...,False,Residential,"10.525053,7.417249",MonoCrystalline,1s2p,10%,1000.0,1.0,Lithium - Ion,NaN
2,5,05/09/2024 00:30,True,05/09/2024 00:30,NaN,21.954235,0.0,0.530610,0.609693,90.0,...,False,Residential,"10.525053,7.417250",MonoCrystalline,1s2p,10%,1000.0,1.0,Lithium - Ion,NaN
3,7,05/09/2024 00:45,True,05/09/2024 00:45,NaN,22.692779,0.0,0.417322,0.477041,90.0,...,False,Residential,"10.525053,7.417251",MonoCrystalline,1s2p,10%,1000.0,1.0,Lithium - Ion,NaN
4,9,05/09/2024 01:00,True,05/09/2024 01:00,NaN,22.200476,0.0,0.314497,0.372710,90.0,...,False,Residential,"10.525053,7.417252",MonoCrystalline,1s2p,10%,1000.0,1.0,Lithium - Ion,NaN


In [3]:
df_try=df.copy()

df_try.drop(columns=['Unnamed: 0','System ON Timestamps','Solar Panels Type',
       'System OFF Timestamps','Inverter Efficiency (%)','System Fault Alerts', 'Voltage (V)', 'Power Factor','Battery Full Flag','Solar Panels Configuration', 'Depth of Discharge',
       'Battery Capacity (Wh)', 'Inverter Capacity (kW)', 'Battery Technology','User Coordinates','System ON','Battery Low Flag','System Load (kW)','Customer Profile', 'Solar Irradiance (W/m²)'], axis=1, inplace=True)

In [4]:
# Set the index of the DataFrame
df_try['Timestamp'] = pd.to_datetime(df_try['Timestamp'],errors='coerce')
df_try.set_index('Timestamp', inplace=True)

In [5]:
df_try.head()

,Temperature (°C),Solar Panels Energy Output (W),Power Consumption (kW),Energy Stored in Batteries (kWh),Current (A),Dust and Dirt Accumulation (g/m²)
Timestamp,,,,,,
2024-05-09 00:00:00,21.027232,0.0,0.232676,0.767498,1.057620,0.001
2024-05-09 00:15:00,21.000000,0.0,0.398544,0.709329,1.811565,0.002
2024-05-09 00:30:00,21.954235,0.0,0.530610,0.609693,2.411864,0.003
2024-05-09 00:45:00,22.692779,0.0,0.417322,0.477041,1.896917,0.004
2024-05-09 01:00:00,22.200476,0.0,0.314497,0.372710,1.429532,0.005


In [6]:
df_try.isna().sum()

Temperature (°C)                     0
Solar Panels Energy Output (W)       0
Power Consumption (kW)               0
Energy Stored in Batteries (kWh)     0
Current (A)                          0
Dust and Dirt Accumulation (g/m²)    0
dtype: int64

In [7]:
#df_try=df_try.resample('h').mean()
df_try.head()

,Temperature (°C),Solar Panels Energy Output (W),Power Consumption (kW),Energy Stored in Batteries (kWh),Current (A),Dust and Dirt Accumulation (g/m²)
Timestamp,,,,,,
2024-05-09 00:00:00,21.027232,0.0,0.232676,0.767498,1.057620,0.001
2024-05-09 00:15:00,21.000000,0.0,0.398544,0.709329,1.811565,0.002
2024-05-09 00:30:00,21.954235,0.0,0.530610,0.609693,2.411864,0.003
2024-05-09 00:45:00,22.692779,0.0,0.417322,0.477041,1.896917,0.004
2024-05-09 01:00:00,22.200476,0.0,0.314497,0.372710,1.429532,0.005


In [9]:

'''
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt

# Normalize the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df_try)

# Define sequence length and features
sequence_length = 24  # Number of time steps in each sequence
num_features = len(df_try.columns)  # Use the number of columns for features

# Create sequences and corresponding labels
sequences = []
labels = []
for i in range(len(scaled_data) - sequence_length):
    seq = scaled_data[i:i + sequence_length]
    label = scaled_data[i + sequence_length, 2]  # 'Power Consumption (kW)' column index
    sequences.append(seq)
    labels.append(label)

# Convert to numpy arrays and then to PyTorch tensors
sequences = torch.tensor(np.array(sequences), dtype=torch.float32)
labels = torch.tensor(np.array(labels), dtype=torch.float32)

# Split into train and test sets
train_size = int(0.8 * len(sequences))
train_x, test_x = sequences[:train_size], sequences[train_size:]
train_y, test_y = labels[:train_size], labels[train_size:]

# Create data loaders
train_dataset = TensorDataset(train_x, train_y)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = TensorDataset(test_x, test_y)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])  # Get the output of the last time step
        return out

# Instantiate the model
input_size = num_features  # Number of features
hidden_size = 64
num_layers = 3
output_size = 1
model = LSTMModel(input_size, hidden_size, num_layers, output_size)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())

# Training loop
epochs = 100
for epoch in range(epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target.unsqueeze(1)) # Reshape target to match output
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

    # Validation (you can add a validation loop here)

# Save the model
torch.save(model.state_dict(), 'pytorch_lstm_model.pth')

# Evaluation
model.eval()
with torch.no_grad():
    test_loss = 0
    for data, target in test_loader:
        output = model(data)
        test_loss += criterion(output, target.unsqueeze(1)).item()  # sum up batch loss

    test_loss /= len(test_loader.dataset)
    print(f'Test set: Average loss: {test_loss:.4f}')
'''

"\nimport torch\nimport torch.nn as nn\nimport torch.optim as optim\nfrom torch.utils.data import DataLoader, TensorDataset\nfrom sklearn.preprocessing import MinMaxScaler\nimport numpy as np\nimport matplotlib.pyplot as plt\n\n# Normalize the data\nscaler = MinMaxScaler()\nscaled_data = scaler.fit_transform(df_try)\n\n# Define sequence length and features\nsequence_length = 24  # Number of time steps in each sequence\nnum_features = len(df_try.columns)  # Use the number of columns for features\n\n# Create sequences and corresponding labels\nsequences = []\nlabels = []\nfor i in range(len(scaled_data) - sequence_length):\n    seq = scaled_data[i:i + sequence_length]\n    label = scaled_data[i + sequence_length, 2]  # 'Power Consumption (kW)' column index\n    sequences.append(seq)\n    labels.append(label)\n\n# Convert to numpy arrays and then to PyTorch tensors\nsequences = torch.tensor(np.array(sequences), dtype=torch.float32)\nlabels = torch.tensor(np.array(labels), dtype=torch.floa

In [10]:
'''
# Visualize Predictions vs Actual values
test_dates = df_try.iloc[train_size + sequence_length:].index
# Convert model predictions to original scale
predictions_original = scaler.inverse_transform(np.concatenate([np.zeros((predictions.shape[0], df_try.shape[1]-1)), predictions.reshape(-1, 1)], axis=1))[:, -1]
actual_values_original = scaler.inverse_transform(np.concatenate([np.zeros((actual_values.shape[0], df_try.shape[1]-1)), actual_values.reshape(-1, 1)], axis=1))[:, -1]

# Plot actual vs predicted power consumption
plt.figure(figsize=(12, 6))

# Plot actual power consumption
plt.plot(test_dates, actual_values_original, label='Actual Power Consumption', color='blue')

# Plot predicted power consumption
plt.plot(test_dates, predictions_original, label='Predicted Power Consumption', color='red', linestyle='--')

# Add labels and title
plt.xlabel('Time')
plt.ylabel('Power Consumption (kW)')
plt.title('Power Consumption: Actual vs Predicted (Test Set)')
plt.legend()

# Show the plot
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
'''

"\n# Visualize Predictions vs Actual values\ntest_dates = df_try.iloc[train_size + sequence_length:].index\n# Convert model predictions to original scale\npredictions_original = scaler.inverse_transform(np.concatenate([np.zeros((predictions.shape[0], df_try.shape[1]-1)), predictions.reshape(-1, 1)], axis=1))[:, -1]\nactual_values_original = scaler.inverse_transform(np.concatenate([np.zeros((actual_values.shape[0], df_try.shape[1]-1)), actual_values.reshape(-1, 1)], axis=1))[:, -1]\n\n# Plot actual vs predicted power consumption\nplt.figure(figsize=(12, 6))\n\n# Plot actual power consumption\nplt.plot(test_dates, actual_values_original, label='Actual Power Consumption', color='blue')\n\n# Plot predicted power consumption\nplt.plot(test_dates, predictions_original, label='Predicted Power Consumption', color='red', linestyle='--')\n\n# Add labels and title\nplt.xlabel('Time')\nplt.ylabel('Power Consumption (kW)')\nplt.title('Power Consumption: Actual vs Predicted (Test Set)')\nplt.legend(

In [13]:
# Set the tracking URI to the local MLflow UI
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import mlflow
import mlflow.pytorch
import optuna
import matplotlib.pyplot as plt

# Normalize the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df_try)

# Define sequence length for 24 hours with 15-minute intervals
sequence_length = 96  # 24 hours * 4 intervals per hour (15-minute intervals)
num_features = len(df_try.columns)  # Use the number of columns for features

# Create sequences and corresponding labels
sequences = []
labels = []
for i in range(len(scaled_data) - sequence_length):
    seq = scaled_data[i:i + sequence_length]
    label = scaled_data[i + sequence_length, 2]  # 'Power Consumption (kW)' column index
    sequences.append(seq)
    labels.append(label)

# Convert to numpy arrays and then to PyTorch tensors
sequences = torch.tensor(np.array(sequences), dtype=torch.float32)
labels = torch.tensor(np.array(labels), dtype=torch.float32)

# Split into train and test sets
train_size = int(0.8 * len(sequences))
train_x, test_x = sequences[:train_size], sequences[train_size:]
train_y, test_y = labels[:train_size], labels[train_size:]

# Create data loaders
train_dataset = TensorDataset(train_x, train_y)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = TensorDataset(test_x, test_y)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])  # Get the output of the last time step
        return out

# Function to train and evaluate the model
def train_and_evaluate(params):
    # Extract hyperparameters from Optuna trial
    hidden_size = params['hidden_size']
    num_layers = params['num_layers']
    learning_rate = params['learning_rate']

    model = LSTMModel(input_size=num_features, hidden_size=hidden_size, num_layers=num_layers, output_size=1)
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    epochs = 100
    for epoch in range(epochs):
        for batch_idx, (data, target) in enumerate(train_loader):
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target.unsqueeze(1))  # Reshape target to match output
            loss.backward()
            optimizer.step()

            if batch_idx % 100 == 0:
                print(f'Epoch {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}]\tLoss: {loss.item():.6f}')
    
    # Evaluation
    model.eval()
    with torch.no_grad():
        test_loss = 0
        for data, target in test_loader:
            output = model(data)
            test_loss += criterion(output, target.unsqueeze(1)).item()  # sum up batch loss

        test_loss /= len(test_loader.dataset)
        print(f'Test set: Average loss: {test_loss:.4f}')
    
    return test_loss

# Optuna Hyperparameter optimization
def objective(trial):
    # Define hyperparameters for optimization
    hidden_size = trial.suggest_int('hidden_size', 32, 128)
    num_layers = trial.suggest_int('num_layers', 1, 4)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)

    # Log the experiment using MLflow
    mlflow.start_run()
    mlflow.log_param('hidden_size', hidden_size)
    mlflow.log_param('num_layers', num_layers)
    mlflow.log_param('learning_rate', learning_rate)

    test_loss = train_and_evaluate({
        'hidden_size': hidden_size,
        'num_layers': num_layers,
        'learning_rate': learning_rate
    })

    mlflow.log_metric('test_loss', test_loss)
    mlflow.end_run()

    return test_loss

# Run the optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

# Best hyperparameters from Optuna
print(f"Best trial: {study.best_trial.params}")

# Load best model and evaluate
best_model = LSTMModel(input_size=num_features, 
                       hidden_size=study.best_trial.params['hidden_size'], 
                       num_layers=study.best_trial.params['num_layers'], 
                       output_size=1)

# Log the model to MLflow
mlflow.pytorch.log_model(best_model, 'best_lstm_model')

# Plot the predictions vs actual values
predictions = []
actual_values = []
test_dates = df_try.iloc[train_size + sequence_length:].index

# Predict and collect results
best_model.eval()
with torch.no_grad():
    for data, target in test_loader:
        output = best_model(data)
        predictions.append(output.numpy())
        actual_values.append(target.numpy())

predictions = np.concatenate(predictions, axis=0)
actual_values = np.concatenate(actual_values, axis=0)

# Inverse transform the predictions and actual values
predictions_original = scaler.inverse_transform(np.concatenate([np.zeros((predictions.shape[0], df_try.shape[1]-1)), predictions.reshape(-1, 1)], axis=1))[:, -1]
actual_values_original = scaler.inverse_transform(np.concatenate([np.zeros((actual_values.shape[0], df_try.shape[1]-1)), actual_values.reshape(-1, 1)], axis=1))[:, -1]

# Plot actual vs predicted power consumption
plt.figure(figsize=(12, 6))
plt.plot(test_dates, actual_values_original, label='Actual Power Consumption', color='blue')
plt.plot(test_dates, predictions_original, label='Predicted Power Consumption', color='red', linestyle='--')
plt.xlabel('Time')
plt.ylabel('Power Consumption (kW)')
plt.title('Power Consumption: Actual vs Predicted (Test Set)')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

[I 2025-01-26 20:04:19,957] A new study created in memory with name: no-name-ec281ca5-0d3c-495c-96a7-bb25a2634cd0
/tmp/ipykernel_3180/1222791714.py:104: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)


Epoch 0 [0/8866]	Loss: 0.102940
Epoch 0 [6400/8866]	Loss: 0.026624
Epoch 1 [0/8866]	Loss: 0.039671
Epoch 1 [6400/8866]	Loss: 0.031028
Epoch 2 [0/8866]	Loss: 0.039339
Epoch 2 [6400/8866]	Loss: 0.028131
Epoch 3 [0/8866]	Loss: 0.027843
Epoch 3 [6400/8866]	Loss: 0.042919
Epoch 4 [0/8866]	Loss: 0.033984
Epoch 4 [6400/8866]	Loss: 0.024272
Epoch 5 [0/8866]	Loss: 0.017209
Epoch 5 [6400/8866]	Loss: 0.022730
Epoch 6 [0/8866]	Loss: 0.018539
Epoch 6 [6400/8866]	Loss: 0.024547
Epoch 7 [0/8866]	Loss: 0.023704
Epoch 7 [6400/8866]	Loss: 0.024170
Epoch 8 [0/8866]	Loss: 0.022399
Epoch 8 [6400/8866]	Loss: 0.030431
Epoch 9 [0/8866]	Loss: 0.036737
Epoch 9 [6400/8866]	Loss: 0.017651
Epoch 10 [0/8866]	Loss: 0.012255
Epoch 10 [6400/8866]	Loss: 0.034396
Epoch 11 [0/8866]	Loss: 0.019235
Epoch 11 [6400/8866]	Loss: 0.018584
Epoch 12 [0/8866]	Loss: 0.015006
Epoch 12 [6400/8866]	Loss: 0.023055
Epoch 13 [0/8866]	Loss: 0.021572
Epoch 13 [6400/8866]	Loss: 0.024121
Epoch 14 [0/8866]	Loss: 0.022523
Epoch 14 [6400/8866]	

[W 2025-01-26 20:05:09,652] Trial 0 failed with parameters: {'hidden_size': 71, 'num_layers': 1, 'learning_rate': 0.0016973307795948335} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/codespace/.python/current/lib/python3.12/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_3180/1222791714.py", line 112, in objective
    test_loss = train_and_evaluate({
                ^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_3180/1222791714.py", line 79, in train_and_evaluate
    loss = criterion(output, target.unsqueeze(1))  # Reshape target to match output
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/codespace/.local/lib/python3.12/site-packages/torch/nn/modules/module.py", line 1736, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/codespace/.l

KeyboardInterrupt: 